In [1]:
%load_ext autoreload
%autoreload 2

In [36]:
# Dependencies
import requests
import pymongo
import dns
import datetime
from datetime import datetime
import pytz
from django.utils import timezone
from time import sleep
import random
import json
import config

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

# Project defined dependencies
import config
from pprint import pprint
# import cl_parser


In [3]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(config.mongo_conn)

In [4]:
# Define database and collection
db = client.getitdone_db
SR_collection = db.sandiego

In [6]:
for SR in SR_collection.find():
    print(SR['service_request_id'])
    break

3190603


In [7]:
SR_collection.find().count()    

<ipython-input-7-3c711ff63af8>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  SR_collection.find().count()


681949

In [10]:
for SR in SR_collection.find():
    print(SR)
    break

{'_id': ObjectId('6062a328659e962620d4dcd6'), 'service_request_id': 3190603, 'date_requested': datetime.datetime(2021, 1, 1, 8, 23), 'case_age_days': 28, 'service_name': 'Container Left Out', 'case_record_type': 'ESD Complaint/Report', 'date_closed': datetime.datetime(2021, 1, 29, 8, 0), 'status': 'Closed', 'lat': 32.749027899999966, 'lng': -117.2405783, 'street_address': '4673 Lotus St', 'council_district': 2, 'comm_plan_code': 23, 'comm_plan_name': 'Ocean Beach', 'case_origin': 'Mobile', 'public_description': '2nd report. Tenant has kept containers out since move in- Nov 1. A warning sticker was placed on container by city approx 3 wks ago. Sticker torn off this week.'}


In [44]:
local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime("2019-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime("2020-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

 
print(dt_start_utc, dt_end_utc)

2019-01-01 08:00:00+00:00 2020-01-01 08:00:00+00:00


In [48]:
filter={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}

result = client['getitdone_db']['sandiego'].find(filter=filter)

In [53]:
i = 0

for SR in result:
    print(SR['service_request_id'])
    # SR['date_requested']
    # SR['case_age_days']
    # SR['service_name']
    # SR['case_record_type']
    # SR['date_closed']
    # SR['status']
    # SR['lat']
    # SR['lng']
    # SR['street_address']
    # SR['council_district']
    # SR['comm_plan_code']
    # SR['comm_plan_name']
    # SR['case_origin']
    # SR['public_description']   

    i += 1
    if i > 100:
        break

2475318
2475319
2475320
2475321
2475322
2475323
2475324
2475325
2475326
2475327
2475328
2475329
2475330
2475331
2475332
2475333
2475334
2475335
2475336
2475337
2475338
2475339
2475340
2475341
2475342
2475343
2475344
2475345
2475346
2475347
2475348
2475349
2475350
2475351
2475352
2475353
2475354
2475355
2475356
2475357
2475358
2475359
2475360
2475361
2475362
2475363
2475364
2475365
2475366
2475367
2475368
2475369
2475370
2475371
2475372
2475373
2475374
2475375
2475376
2475377
2475378
2475379
2475380
2475381
2475382
2475383
2475384
2475385
2475386
2475387
2475388
2475389
2475390
2475391
2475392
2475393
2475394
2475395
2475396
2475397
2475398
2475399
2475400
2475402
2475403
2475404
2475405
2475406
2475407
2475408
2475409
2475410
2475411
2475412
2475413
2475414
2475415
2475416
2475417
2475418
2475419
